In [ ]:
import pandas as pd
API_KEY = ''
SHEET_ID = '1wq-iHEoNbcY1DJIHjkFCTgbQCaRLFwcjJScZs1GJoYI'
TEMPLATE_TAB = 'https://docs.google.com/spreadsheets/d/{SHEET}/gviz/tq?tqx=out:csv&sheet={TAB}'
TEMPLATE_GID = 'https://docs.google.com/spreadsheets/d/{SHEET}/export?format=csv&id={SHEET}&gid={GID}'

In [ ]:
import json
with open('../denticle_codes.json') as f:
    cats = json.load(f)
cats

In [ ]:
codegroups = list(cats.keys())
codegroups

In [ ]:
DENTICLE_PAGES = [
    ('1005933316','Elizabeth_P_596_v0.5'),
    ('978726531','Jean_v0.5'),
    ('1456282775','Anima_Modern_v0.5'),
    ('864364141','Anima_596_v0.5'),
    ('409938244','Elizabeth_P_v0.5'),
    ('1401246793','Isa_v0.5'),
    ('1508930583','Arleth_v0.5'),
    ('2042780634','Nick_denticles_v0.5'),
    ('445379200','Liz_Denticles_V0.5'),
    ('1139734790','Whit_Denticles_V0.5'),
]

In [ ]:
GID,TAB = DENTICLE_PAGES[0]

url = TEMPLATE_GID.format(SHEET=SHEET_ID,GID=GID)
print(url)
df_tab = pd.read_csv(url, skiprows=1)
truncate_cols_until = df_tab.columns.get_loc('Z1.1')-1
keep_cols = list(df_tab.columns)[:truncate_cols_until]
df_tab = df_tab[keep_cols]

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_tab.head(2).T)

In [ ]:
# Trim Descriptions off morpho-group column values
for col in codegroups:
    df_tab[col] = df_tab[col].str.split(n=1,expand=True)[0]
df_tab.head(1)

In [ ]:
df_tab.columns

In [ ]:
FILENAME_COL = 'Full file name'
SAMPLE_PARAMS = []
[SAMPLE_PARAMS.append(col) for col in keep_cols if col not in ['Type',FILENAME_COL]+codegroups]
SAMPLE_PARAMS

In [ ]:
# VAST FILEPATH
BASE='/user/esibert/ichthyolithBase'
TEMPLATE = '{DSDP}/{DSDP}-{P}-{L}-{IODP}-{g}_Hwell_{Nof}_Mcompound_Oflat_I1_TzEDF-0_{X}/{DSDP}-{P}-{L}-{IODP}-{g}_{obj}_edf.tif'
# DSDP-596-P001-L01-1H-2W-5-7cm-g106_Hwell_N1of1_Mcompound_Oflat_I1_TzEDF-0_X5
# DSDP-596-P001-L01-1H-2W-5-7cm-g106_obj00001_edf.tif
IODP = '{H}_{W}_{cm}' # core: 1H, section: 2W, interval: 5-7cm
PID_TEMPLATE = '{DSDP}_{P}_{L}_{IODP}_{g}_{N}_{obj}.tif'

In [ ]:
df_tab[FILENAME_COL].iloc[0]

In [ ]:
# parse filename
# DSDP-596-P001-L01-1H-2W-5-7cm-g106_obj00001_edf.tif
pattern = '(?P<site>DSDP-\d+)-(?P<slide>P\d+)-(?P<sediment>[A-Za-z0-9]+)-(?P<core>\d+H)-(?P<section>\d+W)-(?P<interval>\d+-\d+cm)-(?P<fraction>g\d+)_(?P<obj>obj\d+)_edf.(?P<ext>tif|jpg)'
extracted_columns = df_tab[FILENAME_COL].str.strip().str.extract(pattern)
extracted_columns

In [ ]:
df = pd.concat([df_tab, extracted_columns], axis=1)
print(list(df.columns))

In [ ]:
# drop jpg columns
df = df[df.ext=='tif']

In [ ]:
df['iodp'] = df['core'] + '_' + df['section'] + '_' +df['interval']
df['pid'] = df['site'] + '_' + df['slide'] + '_' + df['sediment'] + '_' + df['iodp'] + '_' +df['fraction'] + '_N' +df['Slide Hole'].astype(int).astype(str) + '_' +df['obj'] 
df.pid.tail()

In [ ]:
# All PIDs properly parsed if is empty
df[df.pid.isna()]

In [ ]:
# Find duplicate entries
dupes1 = df[df[[FILENAME_COL,'Slide Hole']].duplicated(keep=False)]
display(dupes1[['pid',FILENAME_COL, 'Slide Hole']])
for idx,row in dupes1.iterrows():
    print(row[FILENAME_COL])

In [ ]:
# Find duplicate entries
# here this may be caused by there being both .tif and .jpg versions of an image
dupes2 = df[df['pid'].duplicated(keep=False)]
display(dupes2[['pid',FILENAME_COL, 'Slide Hole']])
for idx,row in dupes2.iterrows():
    print(row[FILENAME_COL])

In [ ]:
#If there are any dupes they gotta be cleaned up

In [ ]:
len(df)

In [ ]:
# JPG duplicates
y = dupes2[dupes2.ext=='jpg']
y[y.pid.duplicated(keep=False)]

In [ ]:
# TIFF duplicates
x = dupes2[dupes2.ext=='tif']
x[x.pid.duplicated(keep=False)]

In [ ]:
#df = df.drop(13109)

In [ ]:
# demo only: test malformed row works as intended
#df.loc[1,'A1'] = 'egg'
#df.loc[0,'B3'] = 'nog'

In [ ]:
# ANNOTATION VALIDATION #

In [ ]:
validation = df[['pid']].copy()
for codegroup,codes in cats.items():
    validation[codegroup] = df[codegroup].dropna().apply( lambda x: x not in codes)
validation = validation[validation[cats.keys()].any(axis=1)]
validation_view = validation.replace(False, pd.NA).dropna(axis=1,how='all').fillna('')  # hide Columns with all Nan-or-False
malformed_rows = validation_view.mask(validation==True, df) # show the bad value

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(malformed_rows)

In [ ]:
assert validation_view.empty, 'Validation Failed: Some cell values do not match categories'

In [ ]:
# TODO combine multiple TAB df's to one. 

# generate filepaths

In [ ]:
# VAST FILEPATH
BASE='/user/esibert/ichthyolithBase/'
TEMPLATE = '{DSDP}/{DSDP}-{P}-{L}-{IODP}-{g}_Hwell_{Nof}_Mcompound_Oflat_I1_TzEDF-0_{X}'

## doesn't work on account of not all Slide Holes are present as part of this dataset.
## this causes the Max to not be the true Max, and N1of2 is actually N1of3 and the directory fails
#ofN = df.groupby(FILENAME_COL)['Slide Hole'].max()
#df['Slide Hole Max'] = df[FILENAME_COL].map(ofN)

with open('DSDP-596_dirs.list') as f:
    vast_dirs = f.read().splitlines()
vast_dirs_mapped = {d.split('of')[0]:d for d in vast_dirs}

df['dir'] = df['site'] + '-' + df['slide'] + '-' + df['sediment'] + '-' + \
            df['core'] + '-' + df['section'] + '-' +df['interval'] + '-' +df['fraction'] + \
            '_Hwell_N' + df['Slide Hole'].astype(int).astype(str) 
            #+'of' + df['Slide Hole Max'].astype(str) + '_Mcompount_Oflat_I1_TzEDF-0_X5'

df['dir'] = df.dir.apply(lambda d: vast_dirs_mapped[d])
df['magnification'] = df['dir'].str.split('_').str[-1]
df['Nof'] = df['dir'].str.split('_').str[-6]
df['Slide Hole Max'] = df['Nof'].str.split('of').str[-1].astype(int)

#df['magnification'] = 'X5'
#df.loc[df.dir=='DSDP-596-P022-L44-2H-5W-4-6cm-g106_Hwell_N1of1_Mcompount_Oflat_I1_TzEDF-0','magnification'] = 'X6'
#df.loc[df.dir=='DSDP-596-P023-M02-2H-5W-11-13cm-g106_Hwell_N1of1_Mcompount_Oflat_I1_TzEDF-0','magnification'] = 'X7'
#df.loc[df.dir=='DSDP-596-P023-M03-2H-5W-16-18cm-g106_Hwell_N1of1_Mcompount_Oflat_I1_TzEDF-0','magnification'] = 'X8'
#df.loc[df.dir=='DSDP-596-P024-M04-2H-5W-21-23cm-g106_Hwell_N1of1_Mcompount_Oflat_I1_TzEDF-0','magnification'] = 'X9'
#df.loc[df.dir=='DSDP-596-P024-M05-2H-5W-26-28cm-g106_Hwell_N1of1_Mcompount_Oflat_I1_TzEDF-0','magnification'] = 'X10'

#print(df[df.magnification!='X5']) # (may be None for given Tab dataset)

#df.iloc[1].dir,df.iloc[1].magnification, df.iloc[1].Nof

In [ ]:
df['path'] = BASE + df.site + '/' + df.dir + '/final/focused/' + df[FILENAME_COL]
df.iloc[1].path

In [ ]:
# write to CSV
df.to_csv(f'{TAB}.csv', index=False)

In [ ]:
# TODO write multiple CSVs, synergize their columns